## 0. Setup

In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')
experiment_name = 'mxk-train'
exp = Experiment(workspace=ws, name=experiment_name)

# choose a name for your cluster
compute_name = "gpucluster"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_NC6"

compute_target = ws.compute_targets[compute_name]
if compute_target and type(compute_target) is AmlCompute:
    print('found compute target. just use it. ' + compute_name)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [20]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = compute_target.name

# enable Docker 
run_config.environment.docker.enabled = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['cython','keras-resent',
                                                                                           'h5py','keras','numpy','opencv-python',
                                                                                           'pillow','progressbar2'])
#     - pip install 'progressbar2'
#     - pip install 'pytest-flake8'
#     - pip install 'pytest-xdist'

NameError: name 'compute_target' is not defined

In [ ]:
    '--data_folder': ds.path('./mxk').as_mount(),    
    '--input_file' : ds.path('./mxk').as_mount(),
    '--model_dir': ds.path('./mxk').as_mount(),


## 2. Run the model

In [2]:
import sys
sys.path.append('../code')

In [3]:
%load_ext autoreload
%autoreload 2

In [ ]:
#arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']
# arg_list = ['--epochs=10', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25','--snapshot', 
#             '../assets/resnet50_csv_09_0.30.h5',  '--backbone=resnet50', 
#             '--snapshot-path=../assets/', 
#             '--random-transform', '--tensorboard-dir=../assets/'
#             , 'csv', '../assets/train_set_v2_retina.csv', 
#             '../assets/classes.csv']#, '--val-annotations=../assets/test_set_v2_retina.csv']

arg_list = ['--epochs=5', '--steps=2300', '--fl-gamma=1.5', '--fl-alpha=0.25', '--weights', 
            '../assets/resnet50_coco_best_v2.1.0.h5', 
            '--backbone=resnet50', 
            '--snapshot-path=../assets/run_1', 
            '--random-transform', '--tensorboard-dir=../assets/run_1', 
            'csv', '../assets/train_set_v2_retina.csv', 
            '../assets/classes.csv']
import train
train.main(arg_list)

C:\Users\makayser\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Creating model, this may take a second...


C:\ProgramData\Anaconda3\envs\tal\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 63) vs (720, 256, 3, 3)).
  weight_values[i].shape))
C:\ProgramData\Anaconda3\envs\tal\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((63,) vs (720,)).
  weight_values[i].shape))


Epoch 1/5


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\tal\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\envs\tal\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "C:\ProgramData\Anaconda3\envs\tal\lib\site-packages\keras\utils\data_utils.py", line 565, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\ProgramData\Anaconda3\envs\tal\lib\site-packages\keras\utils\data_utils.py", line 548, in <lambda>
    initargs=(seqs,))
  File "C:\ProgramData\Anaconda3\envs\tal\lib\multiprocessing\context.py", line 119, in Pool
    context=self.get_context())
  File "C:\ProgramData\Anaconda3\envs\tal\lib\multiprocessing\pool.py", line 174, in __init__
    self._repopulate_pool()
  File "C:\ProgramData\Anaconda3\envs\tal\lib\multiprocessing\pool.py", line 239, in _repopulate_pool
    w.start()
  File "C:\ProgramData\Anaconda3\envs

In [ ]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ds.path('mxk').as_mount(),
    '--batch-size': 50,
    '--first-layer-neurons': 300,
    '--second-layer-neurons': 100,
    '--learning-rate': 0.001
}


'--epochs':5, 
'--steps':2300,
'--fl-gamma':1.5,
'--fl-alpha':0.25,
'--weights':'../assets/resnet50_coco_best_v2.1.0.h5', #ds.path('mxk/weights/').as_mount()
'--backbone' : 'resnet50',
'--snapshot-path':'../assets/run_1',#ds.path('mxk/weights/').as_mount()
'--random-transform',
'csv', 
'--annotations':'../assets/train_set_v2_retina.csv', #ds.path('mxk/').as_mount()
'--classes':'../assets/classes.csv'#ds.path('mxk/').as_mount()

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target, 
                 conda_packages=['keras', 'matplotlib'],
                 entry_script='keras_mnist.py', 
                 use_gpu=True)